In [3]:
import numpy
import matplotlib

In [4]:
def sigmoid(x):
    sigmoid = 1/(1 + numpy.exp((-4 * x)))
    return sigmoid

In [5]:
def seq_module(y_t, weights):
    y_t = numpy.insert(y_t, 0, 1)
    y_t = numpy.insert(y_t, 4, 1)
    y_t_plus_one = sigmoid(numpy.dot(weights, y_t))
    return y_t_plus_one

In [26]:
weights = numpy.random.uniform(-1, 1, (3, 5))

In [27]:
y_t = seq_module(y_t, weights)

In [28]:
print y_t

[ 0.98300653  0.01399963  0.00289541]
